# SynthSeg example

In this example we will show how to use the `mri2mesh` package to visualize the voxels and to generate surfaces from a synthetic segmentation. The synthetic segmentation here is assumed to allready be generated using the `SynthSeg` package. The synthetic segmentation is a 3D volume where each voxel has a label. First we do the neccerasy imports and set the pyvista backend to `trame`. 

In [ ]:
from pathlib import Path
import mri2mesh
import numpy as np
import pyvista as pv

pv.set_jupyter_backend('trame')

Note that to get the visualization to show correctly you might need to set the following environment variables

```
export DISPLAY=":99.0"
export PYVISTA_TRAME_SERVER_PROXY_PREFIX="/proxy/"
export PYVISTA_TRAME_SERVER_PROXY_ENABLED="True"
export PYVISTA_OFF_SCREEN=false
export PYVISTA_JUPYTER_BACKEND="html"
```

Next we will try to visualize a Nifty File with a volume clip. Here we will load a allready segmented brain from the dataset https://zenodo.org/records/4899120 (the `ernie` case), which has been segmented with [`SynthSeg`](https://github.com/BBillot/SynthSeg)

In [ ]:
# Path to the Nifty file
path = Path("201_t13d_synthseg.nii.gz")
mri2mesh.viz.volume_clip.main(path)

We could also visualize it as a slice

In [ ]:
mri2mesh.viz.volume_slice.main(path)

Here we visualize all three slices, but you can all specify which axis to show by passing the `axis` keyword (showing all axis correspond to `axis=3`)

In [ ]:
mri2mesh.viz.volume_slice.main(path, axis=2)

To see all the different visalization options you can do

In [ ]:
mri2mesh.viz.list_viz_commands()

You can also visualize `numpy`arrays directly, for example by first loadind the Nifty file with `nibabel`

In [ ]:
import nibabel as nib

img = nib.load(path)
vol = img.get_fdata()

However, you would need to first convert the volume to a vtk object

In [ ]:
img = mri2mesh.vtk_utils.numpy_to_vtkImageData(vol)

Now, you can visualize the image directly

In [ ]:
mri2mesh.viz.volume_clip.volume_clip(img)

The volume contains different labels for the different regions. For example we could plot different slices for the background, which has the label `0`. Let us plot a $5\times 5$ grid for the slices in the $x$-direction

In [ ]:
mask = vol == 0
mri2mesh.viz.mpl_slice.plot_slices(mask, cmap="gray", add_colorbar=True, nx=5, ny=5, slice="y")

Since this particular image is segmented with `SynthSeg` we know the labels for each region. These are also found in 

In [ ]:
labels = mri2mesh.segmentation_labels.SYNTHSEG_LABELS
for k, v in labels.items():
    print(f"{k}: {v}")

## Extracting the surface of the paranchyma

We can first try to extract the parenchyma surface. To do this we first extract all the labels that are not fluid, and then we first remove small objects and then fill inn the small holes

In [ ]:
import skimage.morphology as skim

par_mask = np.logical_not(np.isin(vol, labels["FLUID"]))
par_mask = skim.remove_small_objects(par_mask, mri2mesh.constants.HOLE_THRESHOLD)
par_mask = skim.remove_small_holes(par_mask, mri2mesh.constants.HOLE_THRESHOLD)

We can now plot the slices of the corresponding mask

In [ ]:
mri2mesh.viz.mpl_slice.plot_slices(par_mask, cmap="gray", add_colorbar=True, nx=5, ny=5, slice="z")

We can also extract the isosurface of the mask by first converting the mask to a vtk object and then plotting it with pyvista

In [ ]:
par_mask_img = mri2mesh.vtk_utils.numpy_to_vtkImageData(par_mask.astype(int))
plotter = pv.Plotter()
par_mask_isosurface = par_mask_img.contour([1.0])
plotter.add_mesh_clip_plane(par_mask_isosurface, color="white", show_edges=True)
plotter.show()

Now let us generate the surface using the marching cubes algorithm

In [ ]:
par_surf = mri2mesh.surface.utils.extract_surface(par_mask)

plotter = pv.Plotter()
plotter.add_mesh_clip_plane(par_surf)
plotter.show()

## Extracting surfaces of the lateral ventricles 

Another serfacce we could extract is the surface of the left and right lateral ventricles with label 4 and 43 respectively. You can first plot the slices

In [ ]:
mask_lateral_ventricles = np.logical_or(vol == 4, vol == 43)
mri2mesh.viz.mpl_slice.plot_slices(mask_lateral_ventricles, cmap="gray", add_colorbar=True, nx=5, ny=5, slice="z")

We can also plot the isosurface with pyvista

In [ ]:
mask_lateral_ventricles_img = mri2mesh.vtk_utils.numpy_to_vtkImageData(mask_lateral_ventricles.astype(int))
plotter = pv.Plotter()
surface = mask_lateral_ventricles_img.contour([1.0])
plotter.add_mesh(surface)
plotter.show()

We can now generate a surface of this mask using `pyvista`

In [ ]:
surf_lateral_ventricles = mri2mesh.surface.utils.extract_surface(mask_lateral_ventricles)

Let us plot the surface with `pyvista`

In [ ]:
surf_lateral_ventricles = mri2mesh.surface.utils.extract_surface(mask_lateral_ventricles)
plotter = pv.Plotter()
plotter.add_mesh(surf_lateral_ventricles)
plotter.show()

We see that the surface is not very smooth, but we can use the `smooth_taubin` method to smooth it


In [ ]:
surf_lateral_ventricles_smooth = surf_lateral_ventricles.smooth_taubin(n_iter=20, pass_band=0.05)
plotter = pv.Plotter()
plotter.add_mesh(surf_lateral_ventricles)
plotter.show()